# (1)사전학습, 점진적 학습데이터 확대를 통한 기존 LSTM 모델의 개선

## 19.11.05. 내용 수정
1. 오타 수정(2.1.1. 에서) "10epoch정도에서 0.33%" >>> "10epoch정도에서 33%"

## 목차
- 요약
- 가설 및 실험 내용
- 결과 분석 및 고찰

## 1. 요약
본 "Quick, Draw!" 문제에서는 이미지 분류 문제로서, CIFAR의 이미지 분류문제가 단순히 이미지 정보만 주어진 것에 비해, 
획순정보라는 부가적인 정보가 들어가있어 획순정보의 활용 방식이 기대가 되는 흥미로운 주제입니다.
본 competition의 Notebook에서 stroke 정보를 활용하고자 LSTM을 사용한 
아래의 notebook을 기초로 하여 pretraining 방법과 학습데이터의 점진적 증가를 통해 해당 모델의 성능을 개선하고자 하였습니다.

https://www.kaggle.com/kmader/quickdraw-baseline-lstm-reading-and-submission

LSTM은 sequence에 대하여 적응할 수 있는 모델이기 때문에, 획순 정보를 다루는데 적합한 모델로 파악됩니다.

## 2.1.1. 가설1. 단순한 그림으로 pretraining하면 획순 개념을 학습할 수 있을 것이다.

도형(line.csv, triangle.csv, square.csv, otagon.csv, hexagon.csv, circle.csv, squiggle.csv) 을 구분하는 모델을 pretraining하고
복잡한 그림을 점차 분석해나가는 방식으로 모델을 훈련하면, 모델의 layer를 줄이고 효과적으로 모델을 구현할 수 있지 않을까라는 착안을 하였습니다.

간단하고 명확한 이미지를 갖는 class에 대해서 pretraining을 진행하여 충분히 정확도를 높이고, 
삼각형, 사각형, 원만을 학습데이터로 사용했을 때, early stopping 조건에 의해 10epoch정도에서 33% cat_acc(categorical accuracy)로 학습이 종료되었습니다.
class당 7500 이미지로 하였는데도, 33%를 넘을 수가 없었습니다.

학습데이터에 도형을(직선, 삼각형, 사각형, 오각형, 육각형, 원, 구불구불한 선(squiggle)) 더 추가하고 훈련데이터를 class 당 7500개 이미지로 하여 훈련을 진행했습니다.
이 경우에는 100epoch에서 84.7% cat_acc(85.3% val_cat_acc)로 학습이 종료되었습니다. 따라서, class 종류가 늘어나면 모델의 성능이 개선된 것으로 보여집니다.


340 종의 class를 다시 11 개의 대분류(교통, 사물, 동물 등)로 분류하고, 각 대분류에서 단순한 이미지를 갖는 3가지의 class만 택하여 pretraining을 진행했습니다.
73% cat_acc까지 pretraining을 한 후, 해당시점의 가중치를 저장하여 340 종 모든 class를 학습데이터로 하여 main training을 진행했습니다.

pretraining 후 main training을 시작할 때 cat_acc는 73%에서 0.003%로 감소하였습니다.

## 2.1.2. 가설2. 학습데이터를 점진적으로 늘리면 학습이 잘 될 것이다.

pretraining(32 class, 7500 이미지/class) 이후 main training에서 학습데이터를 340 class를 각 class 당 750개의 이미지로 20 epoch을 학습 후 학습 이미지의 개수를 두 배로(1500개) 늘려서 학습을 진행했습니다.
이 후 20 epoch 당 이미지의 개수를 두 배씩 늘려 학습하는 방식으로 학습을 진행합니다.

pretraining에서 main training으로 넘어갈 때 cat_acc가 73% 에서 3%로 급격히 떨어진 것에 비하여, main training에서는 20 epoch에서 다음 20 epoch 으로 넘어갈 때,
41.7% 에서 35.1%로 정확도가 어느정도 유지되는 것을 볼 수 있습니다. 또 era가 증가할수록 그 감소폭이(-6.6p%, -0.2p%) 줄어들다가 era5로 넘어갈 때는 오히려 증가함을 볼 수 있습니다.

||||||||||
|--- |--- |--- |--- |--- |--- |--- |--- |--- |
|era|era당<br>epoch 수|class 당<br> 이미지 수|첫-마지막<br> cat_acc|첫<br> cat_acc|마지막<br> cat_acc|마지막-마지막<br> cat_acc|epoch당<br> 소요시간|era<br> 총 소요시간|
|era1_pretraining|100|7500|-|0%|73%|-|120s|40m|
|era2_maintraining|20|750|-70p%|3%|41.7%|-31.3p%|120s|40m|
|era3_maintraining|20|1500|-6.6p%|35.1%|49.3%|+7.6p%|180s|60m|
|era4_maintraining|20|3000|-0.2p%|49.1%|56.9%|+7.6p%|6m|120m|      
|era5_maintraining|20|6000|+1.5p%|58.4%|%|p%|12.5m|240m?|      

## 3. 결과분석 및 고찰
era5의 모델로 late submission을 한 결과 0.74238를 획득했습니다.
era5 모델은 총 epoch이 많이 부족하기 때문에, score가 높지 못한 것으로 파악됩니다. 
충분히 많은 epoch을 돌리고, 대조군을 활용하여 비교를 한다면 본 가설에 대해서 더욱 정확한 고찰을 할 수 있습니다.

In [ ]:
cd /kaggle/input/

In [ ]:
pwd

In [ ]:
ls

In [ ]:
ls quickdraw-doodle-recognition/train_simplified/

In [ ]:
ls quickdraw2/

# (2) 340개 class의 11 종 대분류
사물이 161개로 가장 많고, 동물이 53개, 교통이 29개로 많습니다.

In [ ]:
import pandas as pd
df_11_category = pd.read_csv('quickdraw2/340classes_11_category_labeling_libre.csv')
display(df_11_category)

# (3) 340개 class의 이미지 plotting 및 pretraining 데이터로 단순한 그림 찾기

In [ ]:
pwd

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from keras.metrics import top_k_categorical_accuracy
def top_3_accuracy(x,y): return top_k_categorical_accuracy(x,y, 3)
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from glob import glob
import gc
gc.enable()
def get_available_gpus():
    from tensorflow.python.client import device_lib
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

# base 디렉토리와 test 용 csv 파일의 경로를 설정합니다.
base_dir = os.path.join('.', 'quickdraw-doodle-recognition')
test_path = os.path.join(base_dir, 'test_simplified.csv')

In [ ]:
ls

In [ ]:
from ast import literal_eval
ALL_TRAIN_PATHS = glob(os.path.join(base_dir, 'train_simplified', '*.csv'))
COL_NAMES = ['countrycode', 'drawing', 'key_id', 'recognized', 'timestamp', 'word']

def _stack_it(raw_strokes):
    """preprocess the string and make 
    a standard Nx3 stroke vector"""
    stroke_vec = literal_eval(raw_strokes) # string->list
    # unwrap the list
    in_strokes = [(xi,yi,i)  
     for i,(x,y) in enumerate(stroke_vec) 
     for xi,yi in zip(x,y)]
    c_strokes = np.stack(in_strokes)
    # replace stroke id with 1 for continue, 2 for new
    c_strokes[:,2] = [1]+np.diff(c_strokes[:,2]).tolist()
    c_strokes[:,2] += 1 # since 0 is no stroke
    # pad the strokes with zeros
    return pad_sequences(c_strokes.swapaxes(0, 1), 
                         maxlen=STROKE_COUNT, 
                         padding='post').swapaxes(0, 1)

def read_batch(var_l_category_list=[],
               samples=5, 
               start_row=0,
               max_rows = 1000):
    """
    load and process the csv files
    this function is horribly inefficient but simple
    """
    out_df_list = []
    if len(var_l_category_list) != 0:
        for c_path in ALL_TRAIN_PATHS:
            if c_path.split('/')[-1] in var_l_category_list:
                c_df = pd.read_csv(c_path, nrows=max_rows, skiprows=start_row)
                c_df.columns=COL_NAMES
                out_df_list += [c_df.sample(samples)[['drawing', 'word']]]
    elif len(var_l_category_list) == 0:
        for c_path in ALL_TRAIN_PATHS:
            c_df = pd.read_csv(c_path, nrows=max_rows, skiprows=start_row)
            c_df.columns=COL_NAMES
            out_df_list += [c_df.sample(samples)[['drawing', 'word']]]
    full_df = pd.concat(out_df_list)
    full_df['drawing'] = full_df['drawing'].\
        map(_stack_it)
    
    return full_df

In [ ]:
print(len(ALL_TRAIN_PATHS))
print(base_dir)

In [ ]:
word_encoder = LabelEncoder()
def f_preparing_dataset(var_list=[]):
    train_args = dict(var_l_category_list=var_list,
                      samples=TRAIN_SAMPLES, 
                      start_row=0, 
                      max_rows=int(TRAIN_SAMPLES*1.5))
    valid_args = dict(var_l_category_list=var_list,
                      samples=VALID_SAMPLES, 
                      start_row=train_args['max_rows']+1, 
                      max_rows=VALID_SAMPLES+25)
    test_args = dict(var_l_category_list=var_list,
                     samples=TEST_SAMPLES, 
                     start_row=valid_args['max_rows']+train_args['max_rows']+1, 
                     max_rows=TEST_SAMPLES+25)
    train_df, valid_df, test_df = [read_batch(**train_args), read_batch(**valid_args), read_batch(**test_args)]
    
    word_encoder.fit(train_df['word'])
    print('words', len(word_encoder.classes_), '=>', ', '.join([x for x in word_encoder.classes_]))

    time_f = time.time()
    time_d = time_f - time_i
    print("{:2.0f}mins {:2.2f}sec".format(time_d//60, time_d%60))
    return [train_df, valid_df, test_df]

In [ ]:
import time
time_i = time.time()

batch_size = 4096
STROKE_COUNT = 196
TRAIN_SAMPLES = 1    # 이미지 plot을 위해 각 dataset마다 1개 sample만 읽어옵니다.
VALID_SAMPLES = 1    # 이미지 plot을 위해 각 dataset마다 1개 sample만 읽어옵니다.
TEST_SAMPLES = 1     # 이미지 plot을 위해 각 dataset마다 1개 sample만 읽어옵니다.

train_df, valid_df, test_df = f_preparing_dataset()

time_f = time.time()
time_d = time_f - time_i
print("{:2.0f}mins {:2.2f}sec".format(time_d//60, time_d%60))

In [ ]:
def get_Xy(in_df):
    X = np.stack(in_df['drawing'], 0)
    y = to_categorical(word_encoder.transform(in_df['word'].values))
    return X, y
train_X, train_y = get_Xy(train_df)
valid_X, valid_y = get_Xy(valid_df)
test_X, test_y = get_Xy(test_df)
print(train_X.shape)
print(valid_y.shape)

In [ ]:
import time

def f_plot_img_X(var_arr_X, nb_samples, n_img=1, n_class=1):
    time_i = time.time()
    # rand_idxs = np.random.choice(range(train_X.shape[0]), size = 9)
    
    n_tot = n_img * n_class
    n_wide = 10
    n_height = n_tot//10 + (n_tot%10 != 0)
    print(n_tot)
    print(n_height)
    print(n_wide*n_height)
    rand_idxs = np.array([nb_samples*i+j for i in range(n_class) for j in range(n_img) ])
    fig, m_axs = plt.subplots(n_height, n_wide, figsize = (16, int(1.6*n_height)))

    for c_id, c_ax in zip(rand_idxs, m_axs.flatten()):
        test_arr = var_arr_X[c_id]
        test_arr = test_arr[test_arr[:,2]>0, :] # only keep valid points
        lab_idx = np.cumsum(test_arr[:,2]-1)


        for i in np.unique(lab_idx):
            c_ax.plot(test_arr[lab_idx==i,0], 
                    np.max(test_arr[:,1])-test_arr[lab_idx==i,1], '.-')
        c_ax.axis('off')
        c_ax.set_title(word_encoder.classes_[np.argmax(train_y[c_id])])

    plt.show()
    time_f = time.time()
    time_d = time_f - time_i
    print("{:2.0f}mins {:2.2f}sec".format(time_d//60, time_d%60))

### (3).1. 340class 이미지 plotting
340class에 대한 그림을 train_X, test_X, valid_X로부터 plot하였습니다.

In [ ]:
f_plot_img_X(train_X, nb_samples=TRAIN_SAMPLES, n_img=1, n_class=340)
f_plot_img_X(valid_X, nb_samples=VALID_SAMPLES, n_img=1, n_class=340)
f_plot_img_X(test_X, nb_samples=TEST_SAMPLES, n_img=1, n_class=340)

## (4) 사전학습 era1-pretraining(100epoch/7500img/40mins)

In [ ]:
# 11가지의 카테고리로 나누고 각 카테고리에서 3개의 class를 가지고 사전훈련을 진행합니다.
# 11가지 카테고리
# 교통 /사물/ 동물/ 동작/ 음식/ 신체,인물/ 식물/ 과일/ 장소,건물/ 역할/ 개념
# pretraining의 학습데이터는 총 32개 csv 파일
str_11_category_list = \
'''airplane.csv
ambulance.csv
bicycle.csv
alarm clock.csv
anvil.csv
backpack.csv
ant.csv
bat.csv
bear.csv
animal migration.csv
camouflage.csv
snorkel.csv
banana.csv
birthday cake.csv
bread.csv
angel.csv
arm.csv
beard.csv
broccoli.csv
bush.csv
cactus.csv
apple.csv
banana.csv
blueberry.csv
beach.csv
castle.csv
garden.csv
campfire.csv
fire hydrant.csv
pool.csv
circle.csv
hexagon.csv
hurricane.csv'''
l_category_list = str_11_category_list.split('\n')

In [ ]:
# 분당 900 TRAIN_SAMPLES 처리 함
import time

batch_size = 4096
STROKE_COUNT = 196
TRAIN_SAMPLES = 7500
VALID_SAMPLES = 75
TEST_SAMPLES = 50

word_encoder = LabelEncoder()
train_df, valid_df, test_df = f_preparing_dataset(l_category_list)

In [ ]:
from keras.preprocessing import sequence

# main_training 의 모델 구조와 맞추기 위해 340 class 개수로 post padding을 준다.
train_X, train_y = get_Xy(train_df)
train_y = sequence.pad_sequences(train_y, maxlen=340, padding='post')
valid_X, valid_y = get_Xy(valid_df)
valid_y = sequence.pad_sequences(valid_y, maxlen=340, padding='post')
test_X, test_y = get_Xy(test_df)
test_y = np.array(sequence.pad_sequences(test_y, maxlen=340, padding='post'))

print(train_X.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
f_plot_img_X(train_X, nb_samples=TRAIN_SAMPLES, n_img=1, n_class=32)
f_plot_img_X(valid_X, nb_samples=VALID_SAMPLES, n_img=1, n_class=32)
f_plot_img_X(test_X, nb_samples=TEST_SAMPLES, n_img=1, n_class=32)

In [ ]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv1D, LSTM, Dense, Dropout
if len(get_available_gpus())>0:
    # https://twitter.com/fchollet/status/918170264608817152?lang=en
    from keras.layers import CuDNNLSTM as LSTM # this one is about 3x faster on GPU instances
stroke_read_model = Sequential()
# from keras.utils.training_utils import multi_gpu_model


stroke_read_model.add(BatchNormalization(input_shape = (None,)+train_X.shape[2:]))
# filter count and length are taken from the script https://github.com/tensorflow/models/blob/master/tutorials/rnn/quickdraw/train_model.py
stroke_read_model.add(Conv1D(48, (5,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Conv1D(64, (5,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Conv1D(96, (3,)))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(LSTM(128, return_sequences = True))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(LSTM(128, return_sequences = False))
stroke_read_model.add(Dropout(0.3))
stroke_read_model.add(Dense(512))
stroke_read_model.add(Dropout(0.3))

# main trainig 에서 len(word_encoder.classes_)=340 이므로 Dense(340)으로 맞춰준다.
stroke_read_model.add(Dense(340, activation = 'softmax'))

# 멀티 GPU 사용하기
# stroke_read_model = multi_gpu_model(stroke_read_model, gpus=1)
stroke_read_model.compile(optimizer = 'adam', 
                          loss = 'categorical_crossentropy', 
                          metrics = ['categorical_accuracy', top_3_accuracy])
stroke_read_model.summary()

In [ ]:
weight_path="pretrain_weights.best.hdf5".format('stroke_lstm_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)


reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, 
                                   verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
#                       patience=5) # probably needs to be more patient, but kaggle time is limited
                      patience=5*2)
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
history = stroke_read_model.fit(train_X, train_y,
                      validation_data = (valid_X, valid_y), 
                      batch_size = batch_size,
                      epochs = 100,
                      callbacks = callbacks_list,
                      shuffle = True)
stroke_read_model.save_weights('pretrain_weights.hd')

lstm_results = stroke_read_model.evaluate(test_X, test_y, batch_size = 4096)
print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

```
Epoch 20/20
255000/255000 [==============================] - 25s 96us/step - loss: 3.8444 - categorical_accuracy: 0.1638 - top_3_accuracy: 0.3205 - val_loss: 3.5913 - val_categorical_accuracy: 0.2022 - val_top_3_accuracy: 0.3750

Epoch 00020: val_loss improved from 3.72432 to 3.59134, saving model to pretrain_weights.best.hdf5
17000/17000 [==============================] - 1s 35us/step
Accuracy: 20.4%, Top 3 Accuracy 37.4%

Epoch 20/20
255000/255000 [==============================] - 25s 96us/step - loss: 2.2690 - categorical_accuracy: 0.4540 - top_3_accuracy: 0.6680 - val_loss: 2.0096 - val_categorical_accuracy: 0.5108 - val_top_3_accuracy: 0.7212

Epoch 00020: val_loss improved from 2.01422 to 2.00963, saving model to pretrain_weights.best.hdf5
17000/17000 [==============================] - 1s 34us/step
Accuracy: 50.5%, Top 3 Accuracy 71.7%

Epoch 20/20
255000/255000 [==============================] - 25s 96us/step - loss: 2.0509 - categorical_accuracy: 0.5009 - top_3_accuracy: 0.7121 - val_loss: 1.8247 - val_categorical_accuracy: 0.5543 - val_top_3_accuracy: 0.7570

Epoch 00020: val_loss improved from 1.83847 to 1.82466, saving model to pretrain_weights.best.hdf5
17000/17000 [==============================] - 1s 35us/step
Accuracy: 54.7%, Top 3 Accuracy 75.2%

Epoch 20/20
255000/255000 [==============================] - 25s 96us/step - loss: 1.9077 - categorical_accuracy: 0.5315 - top_3_accuracy: 0.7380 - val_loss: 1.7446 - val_categorical_accuracy: 0.5709 - val_top_3_accuracy: 0.7711

Epoch 00020: val_loss did not improve from 1.72911
17000/17000 [==============================] - 1s 35us/step
Accuracy: 56.6%, Top 3 Accuracy 76.5%
```

In [ ]:
df_era1_pretraining = pd.DataFrame(history.history)

## (5).1. main training (era2_maintraining/20epoch/750img/ 41.7%cat_acc/40mins)

In [ ]:
# 분당 30 TRAIN_SAMPLES 처리 함
import time

batch_size = 4096
STROKE_COUNT = 196
TRAIN_SAMPLES = 750
VALID_SAMPLES = int(TRAIN_SAMPLES/10)
TEST_SAMPLES = int(TRAIN_SAMPLES/75*50)

word_encoder = LabelEncoder()
train_df, valid_df, test_df = f_preparing_dataset()

In [ ]:
from keras.preprocessing import sequence

# main_training 의 모델 구조와 맞추기 위해 340 class 개수로 post padding을 준다.
train_X, train_y = get_Xy(train_df)
train_y = sequence.pad_sequences(train_y, maxlen=340, padding='post')
valid_X, valid_y = get_Xy(valid_df)
valid_y = sequence.pad_sequences(valid_y, maxlen=340, padding='post')
test_X, test_y = get_Xy(test_df)
test_y = np.array(sequence.pad_sequences(test_y, maxlen=340, padding='post'))

print(train_X.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
weight_path="era2_maintraining_weights.best.hdf5".format('stroke_lstm_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)


reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, 
                                   verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
#                       patience=5) # probably needs to be more patient, but kaggle time is limited
                      patience=5*2)
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
history2 = stroke_read_model.fit(train_X, train_y,
                      validation_data = (valid_X, valid_y), 
                      batch_size = batch_size,
                      epochs = 20,
                      callbacks = callbacks_list,
                      shuffle = True)
stroke_read_model.save_weights('era2_maintraining_weights.hd')

lstm_results = stroke_read_model.evaluate(test_X, test_y, batch_size = 4096)
print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

In [ ]:
display(pd.DataFrame(history2.history))

## (5).2. main training (era3_maintraining/ 20epoch/ 1500img/ 49.3%cat_acc/ 60m)

In [ ]:
# 분당 60 TRAIN_SAMPLES 처리 함
import time

batch_size = 4096
STROKE_COUNT = 196
TRAIN_SAMPLES = 1500
VALID_SAMPLES = int(TRAIN_SAMPLES/10)
TEST_SAMPLES = int(TRAIN_SAMPLES/75*50)

word_encoder = LabelEncoder()
train_df, valid_df, test_df = f_preparing_dataset()

In [ ]:
from keras.preprocessing import sequence

# main_training 의 모델 구조와 맞추기 위해 340 class 개수로 post padding을 준다.
train_X, train_y = get_Xy(train_df)
train_y = sequence.pad_sequences(train_y, maxlen=340, padding='post')
valid_X, valid_y = get_Xy(valid_df)
valid_y = sequence.pad_sequences(valid_y, maxlen=340, padding='post')
test_X, test_y = get_Xy(test_df)
test_y = np.array(sequence.pad_sequences(test_y, maxlen=340, padding='post'))

print(train_X.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
weight_path="era3_maintraining_weights.best.hdf5".format('stroke_lstm_model')

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min', save_weights_only = True)


reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, 
                                   verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
early = EarlyStopping(monitor="val_loss", 
                      mode="min", 
#                       patience=5) # probably needs to be more patient, but kaggle time is limited
                      patience=5*2)
callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
history3 = stroke_read_model.fit(train_X, train_y,
                      validation_data = (valid_X, valid_y), 
                      batch_size = batch_size,
                      epochs = 20,
                      callbacks = callbacks_list,
                      shuffle = True)
stroke_read_model.save_weights('era3_maintraining_weights.hd')

lstm_results = stroke_read_model.evaluate(test_X, test_y, batch_size = 4096)
print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

In [ ]:
display(pd.DataFrame(history3.history))

## (5).3. main training (era4_maintraining/20epoch/3000img/ 56.9%cat_acc/120mins)

In [ ]:
# # 분당 60 TRAIN_SAMPLES 처리 함
# import time

# batch_size = 4096
# STROKE_COUNT = 196
# TRAIN_SAMPLES = 3000
# VALID_SAMPLES = int(TRAIN_SAMPLES/10)
# TEST_SAMPLES = int(TRAIN_SAMPLES/75*50)

# word_encoder = LabelEncoder()
# train_df, valid_df, test_df = f_preparing_dataset()

In [ ]:
# from keras.preprocessing import sequence

# # main_training 의 모델 구조와 맞추기 위해 340 class 개수로 post padding을 준다.
# train_X, train_y = get_Xy(train_df)
# train_y = sequence.pad_sequences(train_y, maxlen=340, padding='post')
# valid_X, valid_y = get_Xy(valid_df)
# valid_y = sequence.pad_sequences(valid_y, maxlen=340, padding='post')
# test_X, test_y = get_Xy(test_df)
# test_y = np.array(sequence.pad_sequences(test_y, maxlen=340, padding='post'))

# print(train_X.shape)
# print(train_y.shape)
# print(test_y.shape)

In [ ]:
# weight_path="era3_maintraining_weights.best.hdf5".format('stroke_lstm_model')

# checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
#                              save_best_only=True, mode='min', save_weights_only = True)


# reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, 
#                                    verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
# early = EarlyStopping(monitor="val_loss", 
#                       mode="min", 
# #                       patience=5) # probably needs to be more patient, but kaggle time is limited
#                       patience=5*2)
# callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
# history4 = stroke_read_model.fit(train_X, train_y,
#                       validation_data = (valid_X, valid_y), 
#                       batch_size = batch_size,
#                       epochs = 20,
#                       callbacks = callbacks_list,
#                       shuffle = True)
# stroke_read_model.save_weights('era4_maintraining_weights.hd')

# lstm_results = stroke_read_model.evaluate(test_X, test_y, batch_size = 4096)
# print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

In [ ]:
# display(pd.DataFrame(history4.history))

## (5).4. main training (era5_maintraining/20epoch/6000img/ %cat_acc/240mins)

In [ ]:
# # 분당 60 TRAIN_SAMPLES 처리 함
# import time

# batch_size = 4096
# STROKE_COUNT = 196
# TRAIN_SAMPLES = 6000
# VALID_SAMPLES = int(TRAIN_SAMPLES/10)
# TEST_SAMPLES = int(TRAIN_SAMPLES/75*50)

# word_encoder = LabelEncoder()
# train_df, valid_df, test_df = f_preparing_dataset()

In [ ]:
# from keras.preprocessing import sequence

# # main_training 의 모델 구조와 맞추기 위해 340 class 개수로 post padding을 준다.
# train_X, train_y = get_Xy(train_df)
# train_y = sequence.pad_sequences(train_y, maxlen=340, padding='post')
# valid_X, valid_y = get_Xy(valid_df)
# valid_y = sequence.pad_sequences(valid_y, maxlen=340, padding='post')
# test_X, test_y = get_Xy(test_df)
# test_y = np.array(sequence.pad_sequences(test_y, maxlen=340, padding='post'))

# print(train_X.shape)
# print(train_y.shape)
# print(test_y.shape)

In [ ]:
# weight_path="era3_maintraining_weights.best.hdf5".format('stroke_lstm_model')

# checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=1, 
#                              save_best_only=True, mode='min', save_weights_only = True)


# reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=10, 
#                                    verbose=1, mode='auto', epsilon=0.0001, cooldown=5, min_lr=0.0001)
# early = EarlyStopping(monitor="val_loss", 
#                       mode="min", 
# #                       patience=5) # probably needs to be more patient, but kaggle time is limited
#                       patience=5*2)
# callbacks_list = [checkpoint, early, reduceLROnPlat]

In [ ]:
# history5 = stroke_read_model.fit(train_X, train_y,
#                       validation_data = (valid_X, valid_y), 
#                       batch_size = batch_size,
#                       epochs = 20,
#                       callbacks = callbacks_list,
#                       shuffle = True)
# stroke_read_model.save_weights('era5_maintraining_weights.hd')

# lstm_results = stroke_read_model.evaluate(test_X, test_y, batch_size = 4096)
# print('Accuracy: %2.1f%%, Top 3 Accuracy %2.1f%%' % (100*lstm_results[1], 100*lstm_results[2]))

In [ ]:
# display(pd.DataFrame(history5.history))

## (6).1. 분석 (precision, recall, f1-score, support, confusion_matrix)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
test_cat = np.argmax(test_y, 1)
pred_y = stroke_read_model.predict(test_X, batch_size = 4096)
pred_cat = np.argmax(pred_y, 1)
plt.matshow(confusion_matrix(test_cat, pred_cat))
print(classification_report(test_cat, pred_cat, 
                            target_names = [x for x in word_encoder.classes_]))

## (6).2. 분석 (그리는 도중의 모델의 예측)

In [ ]:
points_to_use = [5, 15, 20, 30, 40, 50]
points_to_user = [108]
samples = 12
word_dex = lambda x: word_encoder.classes_[x]
rand_idxs = np.random.choice(range(test_X.shape[0]), size = samples)
fig, m_axs = plt.subplots(len(rand_idxs), len(points_to_use), figsize = (24, samples/8*24))
for c_id, c_axs in zip(rand_idxs, m_axs):
    res_idx = np.argmax(test_y[c_id])
    goal_cat = word_encoder.classes_[res_idx]
    
    for pt_idx, (pts, c_ax) in enumerate(zip(points_to_use, c_axs)):
        test_arr = test_X[c_id, :].copy()
        test_arr[pts:] = 0 # short sequences make CudnnLSTM crash, ugh 
        stroke_pred = stroke_read_model.predict(np.expand_dims(test_arr,0))[0]
        top_10_idx = np.argsort(-1*stroke_pred)[:10]
        top_10_sum = np.sum(stroke_pred[top_10_idx])
        
        test_arr = test_arr[test_arr[:,2]>0, :] # only keep valid points
        lab_idx = np.cumsum(test_arr[:,2]-1)
        for i in np.unique(lab_idx):
            c_ax.plot(test_arr[lab_idx==i,0], 
                    np.max(test_arr[:,1])-test_arr[lab_idx==i,1], # flip y
                      '.-')
        c_ax.axis('off')
        if pt_idx == (len(points_to_use)-1):
            c_ax.set_title('Answer: %s (%2.1f%%) \nPredicted: %s (%2.1f%%)' % (goal_cat, 100*stroke_pred[res_idx]/top_10_sum, word_dex(top_10_idx[0]), 100*stroke_pred[top_10_idx[0]]/top_10_sum))
        else:
            c_ax.set_title('%s (%2.1f%%), %s (%2.1f%%)\nCorrect: (%2.1f%%)' % (word_dex(top_10_idx[0]), 100*stroke_pred[top_10_idx[0]]/top_10_sum, 
                                                                 word_dex(top_10_idx[1]), 100*stroke_pred[top_10_idx[1]]/top_10_sum, 
                                                                 100*stroke_pred[res_idx]/top_10_sum))

## (7).1. Late Submission

In [ ]:
sub_df = pd.read_csv(test_path)
sub_df['drawing'] = sub_df['drawing'].map(_stack_it)

In [ ]:
sub_vec = np.stack(sub_df['drawing'].values, 0)
sub_pred = stroke_read_model.predict(sub_vec, verbose=True, batch_size=4096)

In [ ]:
top_3_pred = [word_encoder.classes_[np.argsort(-1*c_pred)[:3]] for c_pred in sub_pred]

In [ ]:
top_3_pred = [' '.join([col.replace(' ', '_') for col in row]) for row in top_3_pred]
top_3_pred[:3]

## (7).2. 모델의 submission 데이터셋을 통한 예측

In [ ]:
fig, m_axs = plt.subplots(3,3, figsize = (16, 16))
rand_idxs = np.random.choice(range(sub_vec.shape[0]), size = 9)
for c_id, c_ax in zip(rand_idxs, m_axs.flatten()):
    test_arr = sub_vec[c_id]
    test_arr = test_arr[test_arr[:,2]>0, :] # only keep valid points
    lab_idx = np.cumsum(test_arr[:,2]-1)
    for i in np.unique(lab_idx):
        c_ax.plot(test_arr[lab_idx==i,0], 
                np.max(test_arr[:,1])-test_arr[lab_idx==i,1], '.-')
    c_ax.axis('off')
    c_ax.set_title(top_3_pred[c_id])

In [ ]:
pwd

In [ ]:
sub_df['word'] = top_3_pred
sub_df[['key_id', 'word']].to_csv('/kaggle/working/submission.csv', index=False)

In [ ]:
ls /kaggle/working/

In [ ]:
ls

In [ ]:
df_submission = pd.read_csv('submission.csv')
df_submission.head()